In [1]:
%%writefile mapper.py

import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode
f = open("stop_words_en.txt", "r") 
stop_words=f.read().split("\n")
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    uniq_words=Counter([ word.lower() for word in words if word.lower() not in stop_words ])
    total=float(sum(uniq_words.values()))
    for word in uniq_words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%f" %(word+":"+"0",1)            
        print "%s\t%f" %(word+":"+article_id,uniq_words[word]/total)
        #print(word)
            

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key,count = line.strip().split('\t', 1)
        
        
    except ValueError as e:
        continue
    if not key.endswith(":0"):
        print "%s\t%d"%(key,count)
    else:
        count=int(count)
                    
        if current_key != key :
            if current_key:
                print "%s\t%d" % (current_key, word_sum)
        
            word_sum = 0
            current_key = key
            word_sum += count
    if current_key.endswith(":0"):
        print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [4]:
%%writefile mapper_1.py

import sys
import re
for line in sys.stdin:
    try:
        key,count = line.strip().split('\t', 1)
           
    except ValueError as e:
        continue
    if key=="labor:12" or key=="labor:0":
        print "%s\t%s" % (key,count)


Overwriting mapper_1.py


In [ ]:
%%bash

OUT_DIR="wordcount_result_1519019602556817"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null 2>/dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null



In [ ]:
OUT_DIR_1="wordcount_result_1519019602566817"
NUM_REDUCERS=0

hdfs dfs -rm -r -skipTrash ${OUT_DIR_1} > /dev/null 2>/dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper_1.py \
    -mapper "python mapper_1.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR_1} > /dev/null

hdfs dfs -cat ${OUT_DIR1}/part-* | head